### Train

In [ ]:
import os
import random
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler

import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from sklearn import preprocessing

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

print(os.listdir('../data'))
print(os.listdir('../data/train_images'))

In [ ]:
df_train = pd.read_csv('../data/train.csv')
df_train['filepath'] = df_train['label'] + '/' + df_train['image_id']
print(df_train.shape)
df_train.head()

In [ ]:
le = preprocessing.LabelEncoder()
df_train['label'] = le.fit_transform(df_train['label'])
df_train['label'].value_counts()

In [ ]:
df_train.head()

In [ ]:
class PaddyDataset(Dataset):
    def __init__(self, df, labels, transforms=None):
        self.df = df
        self.labels = labels
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        image_id = self.df['filepath'].values[index]
        fpath = f'../data/train_images/{image_id}'
        
        image = cv2.imread(fpath, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            image = self.transforms(image=image)
            image = image['image']

        label = self.labels[index]

        return torch.tensor(image), torch.tensor(label)

In [ ]:
transforms_train = A.Compose([
    A.Resize(224, 224, p=1.0),
    A.Flip(),
    A.RandomBrightnessContrast(p=0.5),
    A.ShiftScaleRotate(p=0.5), 
    A.Normalize(p=1.0),
    ToTensorV2(),                               
])

transforms_valid = A.Compose([   
    A.Resize(224, 224, p=1.0),                   
    A.Normalize(p=1.0),
    ToTensorV2(),
])

In [ ]:
train_image = PaddyDataset(df_train[:1000].reset_index(drop=True), df_train[:1000].reset_index(drop=True)['label'], transforms=transforms_train)

import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
for i in range(2):
    f, axarr = plt.subplots(1, 5)
    for p in range(5):
        idx = np.random.randint(0, len(train_image))
        img, label = train_image[idx]
        axarr[p].imshow(img.transpose(0, 1).transpose(1, 2))
        axarr[p].set_title(label)

In [ ]:
import timm
    
class resnet34(nn.Module):
    def __init__(self):
        super(resnet34, self).__init__()
        
        self.base_model = timm.create_model('resnet34', pretrained=True)
        self.base_model.fc = nn.Linear(in_features=512, out_features=10, bias=True)
        
    def forward(self, x):
        x = self.base_model(x)

        return x

In [ ]:
def train_loop_fn(model, loader, optimizer, loss_func, scheduler, device, epoch, scaler):
    model.train()

    TRAIN_LOSS = []

    bar = tqdm(enumerate(loader), total=len(loader))

    for step, (data, target) in bar:
        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        
        with autocast():
            outputs = model(data)
            loss = loss_func(outputs, target)

            scaler.scale(loss).backward()

            TRAIN_LOSS.append(loss.item())
            smooth_loss = np.mean(TRAIN_LOSS[-30:])
            bar.set_description(f'loss: {loss.item():.5f}, smth: {smooth_loss:.5f}')

            if ((step + 1) % 2 == 0) or ((step + 1) == len(loader)):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

     
        scheduler.step(epoch)

        avg_train_loss = np.mean(TRAIN_LOSS)
    
    return avg_train_loss


def valid_loop_fn(model, loader, loss_func, device):
    model.eval()

    correct = 0.0
    total_samples = 0.0

    VAL_LOSS = []

    bar = tqdm(enumerate(loader), total=len(loader))

    with torch.no_grad():
        for step, (data, target) in bar:

            data = data.to(device)
            target = target.to(device)
            
            outputs = model(data)

            pred = outputs.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            total_samples += data.size()[0]
            loss = loss_func(outputs, target)

            VAL_LOSS.append(loss.item())

            smooth_loss = np.mean(VAL_LOSS[-30:])
            bar.set_description(f'loss: {loss.item():.5f}, smth: {smooth_loss:.5f}')

    avg_valid_loss = np.mean(VAL_LOSS)
    accuracy = 100.0 * correct / total_samples

    return avg_valid_loss, accuracy

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(5, shuffle=True, random_state=42)
df_train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train['label'])):
    df_train.loc[valid_idx, 'fold'] = i
df_train.to_csv('folds.csv', index=False)
df_train.head()

In [ ]:
num_gpu = torch.cuda.device_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = resnet34().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)

scheduler = lr_scheduler.CosineAnnealingLR(optimizer, 5)
loss_func = nn.CrossEntropyLoss()

In [ ]:
folds = df_train.copy()
N_FOLDS = 5

def run(fold):
    
    print(f"Fold: {fold+1} / {N_FOLDS}")

    train_idx = np.where((folds['fold'] != fold))[0]
    valid_idx = np.where((folds['fold'] == fold))[0]

    df_this  = folds.loc[train_idx].reset_index(drop=True)
    df_valid = folds.loc[valid_idx].reset_index(drop=True)

    dataset_train = PaddyDataset(df_this, df_this['label'], transforms=transforms_train)
    dataset_valid = PaddyDataset(df_valid, df_valid['label'], transforms=transforms_valid)

    train_loader = DataLoader(dataset_train, batch_size=16, num_workers=4, shuffle=True)
    valid_loader = DataLoader(dataset_valid, batch_size=32, num_workers=4, shuffle=False)

    kernel_type = 'resnet34'
    best_file = f'./models/{kernel_type}_best_fold{fold}.bin'
    acc_max = float('-inf')

    for epoch in range(5):
        
        scaler = GradScaler()
        avg_train_loss = train_loop_fn(model, train_loader, optimizer, loss_func, scheduler, device, epoch, scaler)     
        avg_valid_loss, accuracy = valid_loop_fn(model, valid_loader, loss_func, device)
      
        content = f"Epoch: {epoch+1} | lr: {optimizer.param_groups[0]['lr']:.7f} | train loss: {avg_train_loss:.4f} | val loss: {avg_valid_loss:.4f} | accuracy: {accuracy:.4f}"
        print(content)

        with open(f'log_{kernel_type}.txt', 'a') as appender:
            appender.write(content + '\n')

        if accuracy > acc_max:
            print('score2 ({:.6f} --> {:.6f}).  Saving model ...'.format(acc_max, accuracy))
            torch.save(model.state_dict(), best_file)
            acc_max = accuracy

        torch.save(model.state_dict(), f'./models/{kernel_type}_final_fold.bin')

In [ ]:
run(0)
run(1)
run(2)
run(3)
run(4)

### Grab OOF

In [ ]:
### Grab OOF

oof = np.zeros((len(df_train), 1)) 

for fold in range(5):
    valid_idx = np.where((folds['fold'] == fold))[0]
    df_valid = folds.loc[valid_idx].reset_index(drop=True)
    dataset_valid = PaddyDataset(df_valid, df_valid['label'], transforms=transforms_valid)
    valid_loader = DataLoader(dataset_valid, batch_size=32, num_workers=4, shuffle=False)
    
    model = resnet34()
    model.load_state_dict(torch.load(f'./models/resnet34_best_fold{fold}.bin'))
    model = model.to(device)
    model.eval()

    bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
    val_preds = torch.zeros((len(valid_idx), 1), dtype=torch.float32, device=device) 

    with torch.no_grad():
        for step, (data, target) in bar:
            data = data.to(device)
            target = target.to(device)
            logit = model(data)
            pred = torch.argmax(logit, dim=1).unsqueeze(1)
            val_preds[step*32:step*32+32] = pred
        oof[valid_idx] = val_preds.cpu().numpy()

In [ ]:
d = {'oof': np.squeeze(oof)}
oof_df = pd.DataFrame(data=d)
oof_df['oof'] = oof_df['oof'].astype(int)
oof_df.to_csv('oof_df_resnet34.csv')
oof_df.head()

In [ ]:
df_train = pd.read_csv('../data/train.csv')
mapping = dict(enumerate(df_train['label'].unique()))
oof_df['oof'] = oof_df['oof'].map(mapping)
oof_df

In [ ]:
df_train['filepath'] = df_train['label'] + '/' + df_train['image_id']
result = pd.concat([oof_df['oof'], df_train[['label', 'image_id', 'filepath']]], axis=1)
result['incorrect_preds'] = result['oof'] != result['label']
wrong_preds_df = result[result['incorrect_preds'] == True]
wrong_preds_df.to_csv('wrong_preds_df.csv', index=False)

In [ ]:
wrong_preds_df.head()

In [ ]:
wrong_preds_df.shape

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(df_train['label'], oof_df['oof'])

In [ ]:
# wrong_preds_df['oof'].value_counts().plot(kind='bar')

In [ ]:
class PaddyDataset(Dataset):
    def __init__(self, df, labels, oof_pred, transforms=None):
        self.df = df
        self.labels = labels
        self.oof_pred = oof_pred
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        image_id = self.df['filepath'].values[index]
        fpath = f'../data/train_images/{image_id}'
        
        image = cv2.imread(fpath, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            image = self.transforms(image=image)
            image = image['image']

        label = self.labels[index]
        oof_pred = self.oof_pred[index]

        return torch.tensor(image), label, oof_pred

In [ ]:
train_image = PaddyDataset(wrong_preds_df.reset_index(drop=True), 
                           wrong_preds_df.reset_index(drop=True)['label'], 
                           wrong_preds_df.reset_index(drop=True)['oof'], 
                           transforms=transforms_valid)

import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
for i in range(2):
    f, axarr = plt.subplots(1, 5)
    for p in range(5):
        idx = np.random.randint(0, len(train_image))
        img, label, oof_pred = train_image[idx]
        axarr[p].imshow(img.transpose(0, 1).transpose(1, 2))
        axarr[p].set_title(f"Label: {label},\nIncorrect Pred: {oof_pred}")

### Testing

In [ ]:
class PaddyTestDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        image_id = self.df['image_id'].values[index]
        fpath = f'../data/test_images/{image_id}'
        
        image = cv2.imread(fpath, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            image = self.transforms(image=image)
            image = image['image']

        return torch.tensor(image)

In [ ]:
data = {'image_id': sorted(os.listdir('../data/test_images/'))}
df_test = pd.DataFrame(data=data)
df_test

In [ ]:
sample = pd.read_csv('../data/sample_submission.csv')

In [ ]:
transforms_test = A.Compose([   
    A.Resize(224, 224, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Normalize(p=1.0),
    ToTensorV2(),
])

In [ ]:
test_image = PaddyTestDataset(df_test[:1000].reset_index(drop=True), transforms=transforms_test)

import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
for i in range(2):
    f, axarr = plt.subplots(1, 5)
    for p in range(5):
        idx = np.random.randint(0, len(test_image))
        img = test_image[idx]
        axarr[p].imshow(img.transpose(0, 1).transpose(1, 2))

In [ ]:
dataset_test = PaddyTestDataset(df_test, transforms=transforms_test)
test_loader = DataLoader(dataset_test, batch_size=32, num_workers=4, shuffle=False)

In [ ]:
TTA = 3
N_FOLDS = 5

def run_tta(fold):
    
    preds = torch.zeros((len(dataset_test), 1), dtype=torch.float32, device=device) 
    
    model = resnet34()
    model.load_state_dict(torch.load(f'./models/resnet34_best_fold{fold}.bin'))
    model = model.to(device)

    tta_preds = torch.zeros((len(dataset_test), 1), dtype=torch.float32, device=device)

    with torch.no_grad():
        for tta_fold in range(TTA):
            print(f'TTA: {tta_fold}')
            for i, batch in tqdm(enumerate(test_loader), total=len(test_loader)):
                image = batch.to(device, dtype=torch.float32)
                logit = model(image)
                pred = torch.argmax(logit, dim=1).unsqueeze(1)
                tta_preds[i*32:i*32+32] += pred
        preds += tta_preds // TTA
    
    return preds

In [ ]:
fold0_tta = run_tta(0)
fold1_tta = run_tta(1)
fold2_tta = run_tta(2)
fold3_tta = run_tta(3)
fold4_tta = run_tta(4)    

In [ ]:
# 0 - vertical, 1 - horizontal
# horizontally concat. preds per fold 
horizontal_stack = torch.cat((fold0_tta, fold1_tta, fold2_tta, fold3_tta, fold4_tta), 1)
print(horizontal_stack.shape)
print(horizontal_stack)

In [ ]:
# select the most frequently occurring (mode) pred and use that
values, indices = torch.mode(horizontal_stack, dim=1)

In [ ]:
sample['label'] = values.cpu().detach().numpy().astype(int)
sample.head()

In [ ]:
sample['label'].value_counts()

In [ ]:
df_train = pd.read_csv('../data/train.csv')

In [ ]:
mapping = dict(enumerate(df_train['label'].unique()))
mapping

In [ ]:
sample['label'] = sample['label'].map(mapping)

In [ ]:
sample.head()

In [ ]:
sample.to_csv('submission.csv', index=False)

In [ ]:
! kaggle competitions submit -f submission.csv -m 'resnet34 baseline 5fold 3tta' paddy-disease-classification

### Pseudo-Label

In [ ]:
df_train = pd.read_csv('../data/train.csv')
df_train["pl"] = np.zeros_like(df_train["image_id"])
# df_train['filepath'] = df_train['label'] + '/' + df_train['image_id']
print(df_train.shape)
df_train.head()

In [ ]:
df_train = pd.concat([df_train, sample]).reset_index()
print(df_train.shape)
df_train.head()

In [ ]:
df_train.tail()

In [ ]:
df_train['filepath'] = df_train['label'] + '/' + df_train['image_id']
df_train.head()

In [ ]:
df_train.tail()

In [ ]:
le = preprocessing.LabelEncoder()
df_train['label'] = le.fit_transform(df_train['label'])
df_train['label'].value_counts()

In [ ]:
df_train.head()

In [ ]:
class PaddyDataset2(Dataset):
    def __init__(self, df, labels, transforms=None):
        self.df = df
        self.labels = labels
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        image_id = self.df['filepath'].values[index]
        test_image_id = self.df['image_id'].values[index]
        # fpath = f'../data/train_images/{image_id}'
        dir = self.df['pl'].values[index]
        
        if dir:
            fpath = f'../data/test_images/{test_image_id}'
        else:
            fpath = f'../data/train_images/{image_id}'
        
        image = cv2.imread(fpath, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            image = self.transforms(image=image)
            image = image['image']

        label = self.labels[index]

        return torch.tensor(image), torch.tensor(label)

In [ ]:
train_image = PaddyDataset2(df_train[:1000].reset_index(drop=True), df_train[:1000].reset_index(drop=True)['label'], transforms=transforms_train)

import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
for i in range(2):
    f, axarr = plt.subplots(1, 5)
    for p in range(5):
        idx = np.random.randint(0, len(train_image))
        img, label = train_image[idx]
        axarr[p].imshow(img.transpose(0, 1).transpose(1, 2))
        axarr[p].set_title(label)

In [ ]:
# Don't re-reun this cell use the already existing folds.csv file
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(5, shuffle=True, random_state=42)
df_train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train['label'])):
    df_train.loc[valid_idx, 'fold'] = i
df_train.to_csv('folds.csv', index=False)
df_train.head()

In [ ]:
num_gpu = torch.cuda.device_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = resnet34().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)

scheduler = lr_scheduler.CosineAnnealingLR(optimizer, 5)
loss_func = nn.CrossEntropyLoss()

In [ ]:
folds = df_train.copy()
N_FOLDS = 5

def run(fold):
    
    print(f"Fold: {fold+1} / {N_FOLDS}")

    train_idx = np.where((folds['fold'] != fold))[0]
    valid_idx = np.where((folds['fold'] == fold))[0]

    df_this  = folds.loc[train_idx].reset_index(drop=True)
    df_valid = folds.loc[valid_idx].reset_index(drop=True)

    dataset_train = PaddyDataset2(df_this, df_this['label'], transforms=transforms_train)
    dataset_valid = PaddyDataset2(df_valid, df_valid['label'], transforms=transforms_valid)

    train_loader = DataLoader(dataset_train, batch_size=16, num_workers=4, shuffle=True)
    valid_loader = DataLoader(dataset_valid, batch_size=32, num_workers=4, shuffle=False)

    kernel_type = 'resnet34_pl'
    best_file = f'./models/{kernel_type}_best_fold{fold}.bin'
    acc_max = float('-inf')

    for epoch in range(5):
        
        scaler = GradScaler()
        avg_train_loss = train_loop_fn(model, train_loader, optimizer, loss_func, scheduler, device, epoch, scaler)     
        avg_valid_loss, accuracy = valid_loop_fn(model, valid_loader, loss_func, device)
      
        content = f"Epoch: {epoch+1} | lr: {optimizer.param_groups[0]['lr']:.7f} | train loss: {avg_train_loss:.4f} | val loss: {avg_valid_loss:.4f} | accuracy: {accuracy:.4f}"
        print(content)

        with open(f'log_{kernel_type}.txt', 'a') as appender:
            appender.write(content + '\n')

        if accuracy > acc_max:
            print('score2 ({:.6f} --> {:.6f}).  Saving model ...'.format(acc_max, accuracy))
            torch.save(model.state_dict(), best_file)
            acc_max = accuracy

        torch.save(model.state_dict(), f'{kernel_type}_final_fold.bin')

In [ ]:
run(0)
run(1)
run(2)
run(3)
run(4)

### Testing Pseudo-Label

In [ ]:
class PaddyTestDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        image_id = self.df['image_id'].values[index]
        fpath = f'../data/test_images/{image_id}'
        
        image = cv2.imread(fpath, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            image = self.transforms(image=image)
            image = image['image']

        return torch.tensor(image)

In [ ]:
data = {'image_id': sorted(os.listdir('../data/test_images/'))}
df_test = pd.DataFrame(data=data)
df_test

In [ ]:
sample = pd.read_csv('../data/sample_submission.csv')

In [ ]:
transforms_test = A.Compose([   
    A.Resize(224, 224, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Normalize(p=1.0),
    ToTensorV2(),
])

In [ ]:
TTA = 3
N_FOLDS = 5

def run_tta(fold):
    
    preds = torch.zeros((len(dataset_test), 1), dtype=torch.float32, device=device) 
    
    model = resnet34()
    model.load_state_dict(torch.load(f'./models/resnet34_pl_best_fold{fold}.bin'))
    model = model.to(device)

    tta_preds = torch.zeros((len(dataset_test), 1), dtype=torch.float32, device=device)

    with torch.no_grad():
        for tta_fold in range(TTA):
            print(f'TTA: {tta_fold}')
            for i, batch in tqdm(enumerate(test_loader), total=len(test_loader)):
                image = batch.to(device, dtype=torch.float32)
                logit = model(image)
                pred = torch.argmax(logit, dim=1).unsqueeze(1)
                tta_preds[i*32:i*32+32] += pred
        preds += tta_preds // TTA
    
    return preds

In [ ]:
fold0_tta = run_tta(0)
fold1_tta = run_tta(1)
fold2_tta = run_tta(2)
fold3_tta = run_tta(3)
fold4_tta = run_tta(4)

In [ ]:
# 0 - vertical, 1 - horizontal
# horizontally concat. preds per fold 
horizontal_stack = torch.cat((fold0_tta, fold1_tta, fold2_tta, fold3_tta, fold4_tta), 1)
print(horizontal_stack.shape)
print(horizontal_stack)

In [ ]:
horizontal_stack[-3:]

In [ ]:
values[-3:]

In [ ]:
# select the most frequently occurring (mode) pred and use that
values, indices = torch.mode(horizontal_stack, dim=1)

In [ ]:
sample['label'] = values.cpu().detach().numpy().astype(int)
sample.head()

In [ ]:
sample['label'].value_counts()

In [ ]:
_df_train = pd.read_csv('../data/train.csv')
_df_train.shape

In [ ]:
mapping = dict(enumerate(_df_train['label'].unique()))
mapping

In [ ]:
sample['label'] = sample['label'].map(mapping)

sample.to_csv('submission.csv', index=False)

In [ ]:
sample

In [ ]:
! kaggle competitions submit -f submission.csv -m 'resnet34 pseudo label baseline 5fold 3tta' paddy-disease-classification